## Projeto
Classificação binária - Real ou Deepfake.

---

Link do desafio no Kaggle:
https://www.kaggle.com/competitions/image-classification-real-or-ai-generated-photo/data

In [ ]:
# Abrir dataset

# Dividir em treino e validação (teste esta separado)

# Imprimir imagens das flores

In [ ]:
# (pré-processamento, normalização, validação, modelagem, comparação).